<a href="https://colab.research.google.com/github/Psychohistorian11/BitBusters/blob/master/TallerOrdenamiento/Taller_de_Ordenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTANTE

## Debes ejecutar TODO el Notebook

### Lo puedes hacer así: Entorno de ejecución -> Ejecutar todo

#### Al final del Notebook encontrarás la consola de interacción.

* Counting Sort funciona para números enteros
* Radix Sort sólo para números naturales
* El resto funciona para los Reales

In [17]:
def MergeSort(lista_izquierda, lista_derecha):
  lista_resultado = []

  while(len(lista_izquierda) > 0 and len(lista_derecha) > 0):
    if lista_izquierda[0] < lista_derecha[0]:
      lista_resultado.append(lista_izquierda[0])
      lista_izquierda = lista_izquierda[1:]
    else:
      lista_resultado.append(lista_derecha[0])
      lista_derecha = lista_derecha[1:]

  if len(lista_derecha) > 0:
    lista_resultado = lista_resultado + lista_derecha
  if len(lista_izquierda) > 0:
    lista_resultado = lista_resultado + lista_izquierda

  return lista_resultado

def Merge(lista):
  # Caso Base
  largo = len(lista)
  if largo <= 1:
    return lista
  # Dividir el arreglo
  lista_izq = lista[:len(lista)//2]
  lista_der = lista[len(lista)//2:]

  lista_izq = Merge(lista_izq)
  lista_der = Merge(lista_der)

  return MergeSort(lista_izq,lista_der)

In [18]:
def Quick(lista):
  # Caso Base
  largo = len(lista)
  if largo <= 1:
    return lista

  pivote = lista.pop()
  lista1 = []
  lista2 = []

  for i in lista:
    if i <= pivote:
      lista1.append(i)
    else:
      lista2.append(i)

  lista1 = Quick(lista1)
  lista2 = Quick(lista2)

  return lista1 + [pivote] + lista2

In [19]:
def Counting(arr):
    numero_menor = min(arr)
    numero_mayor = max(arr)
    arreglo_contador = [0 for _ in range(numero_menor, numero_mayor+1)]
    arreglo_final = [0 for _ in range(0,len(arr))]
    for numero in arr:
        if numero_menor <0:
            if numero < 0:
                arreglo_contador[(numero_menor*-1)+numero]+=1
            elif numero ==0:
                arreglo_contador[(numero_menor * -1)] += 1
            elif numero >0:
                arreglo_contador[(numero_menor* -1)+numero]+=1
        if numero_menor>=0:
            arreglo_contador[numero-numero_menor]+=1

    for i in range(len(arreglo_contador)-1):
        arreglo_contador[i+1] = arreglo_contador[i]+arreglo_contador[i+1]
    for numero in arr:
        if numero_menor<0:
            if numero <0:
                arreglo_final[arreglo_contador[(numero_menor*-1)+numero]-1]=numero
                arreglo_contador[(numero_menor*-1)+numero] -=1
            elif numero == 0:
                arreglo_final[arreglo_contador[(numero_menor *-1)]-1]=numero
                arreglo_contador[(numero_menor*-1)] -= 1
            elif numero >0:
                arreglo_final[arreglo_contador[(numero_menor* -1)+numero]-1]=numero
                arreglo_contador[(numero_menor*-1+numero)] -=1
        if numero_menor >=0:
            arreglo_final[arreglo_contador[numero-numero_menor]-1]=numero
            arreglo_contador[numero-numero_menor]-=1
    return arreglo_final

In [20]:
def Bucket(lista):
  max_lista = max(lista)
  min_lista = min(lista)

  rango=0
  if True:
    rango= (max_lista-min_lista)/(len(lista)-1)
  else:
    rango = 1

  buckets = [[] for _ in range(len(lista))]

  for i in lista:
    index = int((i-min_lista)/rango)
    if index == len(lista):
      index -= 1
    buckets[index].append(i)

  resultado = []
  for bucket in buckets:
    bucket = Quick(bucket)
    resultado.extend(bucket)

  return resultado

In [21]:
def heapify(arr, n, i):
    mayor = i
    izquierda = 2 * i + 1
    derecha = 2 * i + 2

    if izquierda < n and arr[izquierda] > arr[mayor]:
        mayor = izquierda

    if derecha < n and arr[derecha] > arr[mayor]:
        mayor = derecha

    if mayor != i:
        arr[i], arr[mayor] = arr[mayor], arr[i]
        heapify(arr, n, mayor)

def Heap(arr):
    n = len(arr)

    for i in range(n // 2 - 1, -1, -1):
        heapify(arr, n, i)

    for i in range(n - 1, 0, -1):
        arr[i], arr[0] = arr[0], arr[i]
        heapify(arr, i, 0)

    return arr

In [22]:
def counting_s(arr, exp):
    n = len(arr)
    output = [0] * n
    count = [0] * 10

    for i in range(n):
        index = (arr[i] // exp)
        count[index % 10] += 1

    for i in range(1, 10):
        count[i] += count[i - 1]

    i = n - 1
    while i >= 0:
        index = (arr[i] // exp)
        output[count[index % 10] - 1] = arr[i]
        count[index % 10] -= 1
        i -= 1

    i=0
    for i in range(len(arr)):
        arr[i] = output[i]

def Radix(arr):
    max_element = max(arr)

    exp = 1
    while max_element // exp >= 1:
        counting_s(arr, exp)
        exp *= 10
    print(arr)
    return arr

In [39]:
import pandas as pd
import json
import requests
from pandas import json_normalize
import ipywidgets as widgets
from IPython.display import clear_output
import copy

dict_column_index = {'nit':0, 'ciiu':6, 'ingresos_operacionales':8,
          "ganancia_p_rdida":9,"total_activos":10,"total_pasivos":11,"total_patrimonio":12,
          "a_o_de_corte":13}

dict_sort_function = {
    'Merge Sort': Merge,
    'Quick Sort': Quick,
    'Heap Sort': Heap,
    'Counting Sort': Counting,
    'Radix Sort': Radix,
    'Bucket Sort': Bucket,
}

def obtener_dataf_columnas_numericas():
  link = "https://www.datos.gov.co/resource/6cat-2gcs.json"

  datos = requests.get(link)
  data = json.loads(datos.text)
  data = json_normalize(data)

  for col in dict_column_index.keys():
    data[col] = pd.to_numeric(data[col].str.replace('$', ''), errors='coerce')
    clear_output()
  return data

def ordenar(m_original, columna_ordenada, nombre_columna):
  m_definitiva = []
  copia_m_original = copy.deepcopy(m_original)
  indice_columna = dict_column_index[nombre_columna]
  for i in range(len(columna_ordenada)):
    for j in range(len(columna_ordenada)):
        if copia_m_original[j][indice_columna] == columna_ordenada[i]:
          m_definitiva.append(copy.deepcopy(m_original[j]))
          m_definitiva[-1][indice_columna] = "---------->" +str(copia_m_original[j][indice_columna])+ "<----------"
          copia_m_original[j][indice_columna] = None
  return m_definitiva

d = obtener_dataf_columnas_numericas()
while True:
  print("Si en algun momento no aparece el cuadro de texto, por favor inicie de nuevo el programa")
  print("Al momento de ver los datos, la columna resaltada con flechas ----> <---- es la columna por la que se ordenaron los datos")
  print("Elige columna:\n - 0:nit - 1:ciiu - 2:ingresos_operacionales \n - 3:ganancia_p_rdida - 4:total_activos - 5:total_pasivos \n - 6:total_patrimonio - 7:a_o_de_corte")
  index_col = int(input())
  columns = list(dict_column_index.keys())
  column_name = columns[index_col]

  d = obtener_dataf_columnas_numericas()
  print(column_name)
  print("Elige ordenamiento:\n - 0:Merge - 1:Quick - 2:Heap - 3:Counting(para int positivos y negativos) \n - 4:Radix(solo int positivos) - 5:Bucket")
  index_sort = int(input())
  sorts = list(dict_sort_function.keys())
  sort_name = sorts[index_sort]
  ordenamientos_enteros = ["Counting Sort", "Radix Sort"]
  lista_columnas_valores_negativos = ["ganancia_p_rdida","total_patrimonio"]
  if sort_name == "Radix Sort" and column_name in lista_columnas_valores_negativos:
    print("No puedes ordenar una columna con valores negativos con Radix Sort")
    continue # SALTA LA ITERACIÓN (NO EJECUTA LO QUE ESTÁ DEBAJO)
  l_ord= []
  if sort_name in ordenamientos_enteros:
    d[column_name] = d[column_name].astype(int)
    l_ord = dict_sort_function[sort_name](d[column_name].tolist())
  else:
    #d[column_name] = pd.to_numeric(d[column_name])
    l_ord = dict_sort_function[sort_name](d[column_name].tolist())
  matriz = d.values.tolist()
  m = ordenar(matriz, l_ord, column_name)
  for i in range(len(m)):
    print(m[i])

Si en algun momento no aparece el cuadro de texto, por favor inicie de nuevo el programa
Al momento de ver los datos, la columna resaltada con flechas ----> <---- es la columna por la que se ordenaron los datos
Elige columna:
 - 0:nit - 1:ciiu - 2:ingresos_operacionales 
 - 3:ganancia_p_rdida - 4:total_activos - 5:total_pasivos 
 - 6:total_patrimonio - 7:a_o_de_corte


KeyboardInterrupt: ignored